In [4]:
from dataclasses import dataclass
from math import gcd, ceil
import re
from collections import Counter, defaultdict, namedtuple, deque
from itertools import product, chain, combinations, combinations_with_replacement, permutations, accumulate, zip_longest
import numpy as np
from matplotlib import pyplot as plt
import networkx as nx

from aocutils.common import copy_func, patch_to, patch, to_int, ints, flatten, zippify, multidict, rev, data, quantify, atom, atoms, list_atoms, list_multiply, mapt
from aocutils.grid import gridneigh, arr_to_dict, grid_to_dict, neighbors, arr_neighbors, iterate, Dim, dimensions, positive, manhattan, conv1d, conv2d, rotate
from aocutils.maze import bfs, dijkstra, get_path, dfs
from aocutils.math import gcd, factors, lcm, crt, mul_inv, Segment, lis, angle, all_combinations, all_permutations, mst
from aocutils.special import md5, binarysearch, deduce_matches, find_specific_pattern, find_repeat, find_cycle, UnionFind, Octree, LazySegmentTree, Trie
from aocutils.visuals import visualize_graph, labelize, animate_grid, plot
from aocutils.cfg import CFG
from aocutils.earley import State, Earley
from aocutils.shunting import ShuntingYard

In [33]:
f = open("in.txt").read().split("\n")
lines = [ints(line) for line in f]
taken = set()
sensors = set()
ans = 0
goal = set()
tar = 2000000
for x,y,x2,y2 in lines:
    sensors.add((x,y))
    taken.add((x,y))
    distance = abs(x2-x) + abs(y2-y)
    deltatar = abs(tar - y)
    leftdistance = distance - deltatar
    if leftdistance > 0:
        ans += leftdistance
        for i in range(x,x+leftdistance):
            goal.add((i,tar))
        for i in range(x-leftdistance,x+1):
            goal.add((i,tar))
len(goal)

    
    

5176944

In [34]:
goal.pop()

(1882733, 2000000)

In [12]:
#| export

class LazySegmentTree():
    # based on https://leetcode.com/problems/my-calendar-iii/solution/
    # lazy: the number of events covering all times in the range. As all numbers that belong to this range will be added by some increment, we don't have to propagate the base increment to every time in the interval, all we need to do is putting the number in this lazy field. 
    # We only update val by adding lazy when requested to query the max numbers of intervals in [L, R].
    def __init__(self, left, right) -> None:
        self.left, self.right = left, right # detailing the range of interest
        self.idxs = defaultdict(int)
        self.lazy = defaultdict(int)
        
    def add(self, start, end, l=None,r=None, idx=1):
        if l == None:
            l = self.left
        if r == None:
            r = self.right
        if l > end or r < start:
            return
        elif l >= start and r <= end and l <=r:
            self.idxs[idx] += 1
            self.lazy[idx] += 1
        else:
            mid = (l + r) // 2
            self.add(start, end, l, mid, idx*2)
            self.add(start, end, mid+1, r, idx*2 + 1)
            self.idxs[idx] = self.lazy.get(idx,0) + min((self.idxs.get(idx*2,0),self.idxs.get(idx*2+1,0)))
            
    def getbest(self):
        return self.idxs[1]

In [19]:
f = open("in.txt").read().split("\n")
lines = [ints(line) for line in f]
taken = set()
sensors = set()
ans = 0
goal = set()
tar = 2000000
seg = LazySegmentTree(0,4000000)
segs = [LazySegmentTree(0,4000000) for i in range(4000001)]
rows = 4000001
for x,y,x2,y2 in lines:
    print(x)
    distance = abs(x2-x) + abs(y2-y)
    for i in range(rows):
        deltatar = abs(i - y)
        leftdistance = distance - deltatar
        if leftdistance > 0:
            seg.add(min(4000000, x-leftdistance), max(0,x+leftdistance))
    
len(goal)

    

3999724
3995530
3016889
3443945
168575
2820722
2329102
3889469
3256726
3729564
206718
2178192
1884402
3060435
3998355
3704399
1421672
3415633
2408019
3735247
1756494
780161
3036853
53246
2110517
3149491


0

In [20]:
1+1

2

In [23]:
for i, s in enumerate(segs):
    if not s.getbest():
        print(i)
        tar = i
        break

0


In [25]:
for i, s in enumerate(segs[:10]):
    print(s.getbest())

0
0
0
0
0
0
0
0
0
0


In [9]:
f = open("t0.txt").read().split("\n")
lines = [ints(line) for line in f]
taken = set()
sensors = set()
goal = set()
upper = 20
for i in range(0,upper):
    
    seg = []
    for x,y,x2,y2 in lines:
        distance = abs(x2-x) + abs(y2-y)
        deltatar = abs(i - y)
        leftdistance = distance - deltatar
        if leftdistance > 0:
            seg.append((max(0,x-leftdistance), min(upper, x+leftdistance)))
    seg.sort()
    found = 0
    maxx = 0
    for x,y in seg:
        if x > maxx+1:
            print('found',x,i)
        maxx = max(maxx, y)
        if maxx > upper:
            break
        



found 15 11


In [10]:
3337614*4000000+ 2933732

13350458933732

In [ ]:
4000000

In [7]:
2933732*4000000+3337615 

11734931337615

In [ ]:
13350462933732
11734931337615

In [ ]:
for i in range(rows):
    if (i,tar) not in goal:
        print(i,tar)
        print(i*4000000+tar)

In [15]:
seg.getbest()

1

In [3]:
grid, neigh = gridneigh("in.txt", to_dict=True, parser=lambda x: [int(ch) for ch in x])
conn = graphparse("in.txt", parser=lambda line: [x for x in line.split()])
conn = graphparse(
    "in.txt",
    parser=lambda line: [int(x) if x.isnumeric() else x for x in line.split()],
    weights=True,
)

ValueError: invalid literal for int() with base 10: 'T'